In [29]:
import requests 
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from multiprocessing import Pool

## Henter url fra foretaksregisteret

In [4]:
url = "https://hotell.difi.no/api/csv/brreg/enhetsregisteret?"

In [11]:
df = pd.read_csv(url, sep=';', encoding='utf-8')
df = df[df.url.notna()]

In [19]:
urls = df['url'].unique().tolist()

In [20]:
urls

['www.mayasart.net',
 'www.ugandasbarn.org',
 'www.arnestad.org',
 'mikkelcappelensmith.com',
 'www.plussark.no',
 'www.pluss2.no',
 'palmera-i-bergen.no',
 'www.ebe-data.com',
 'www.soldier.no',
 'artofdesign.ltevensen.com',
 'www.artific.no',
 'www.01kommunikasjon.no',
 'www.travsport.no/Organisasjon/ForbundLag/Hedmark-Travforbund/Travlag/Grue-Travklubb/',
 'www.07.no',
 'www.getpersonal.no',
 'www.08aerosystem.se',
 'www.12x.no',
 'www.1aid.no']

In [39]:
def getContent(url):
    response = requests.get(f'http://{url}')
    if response.ok:
        return response.text
    response = requests.get(f'https://{url}')
    if response.ok:
        return response.text

In [40]:
print(getContent(urls[0]))



<script type="text/javascript">
    var thisDomain = '';
    if (thisDomain.length > 0) {
        document.domain = thisDomain;
    }
</script>
<!DOCTYPE html>
<html lang="nb-NO">
    <head>
        <title>Mayasart - www.mayasart.net</title>
<meta property="fb:app_id" content="1880640628839943" />
<meta property="og:site_name" content="Mayasart" />
<meta property="article:publisher" content="https://www.facebook.com/simplesite" />
<meta property="og:locale" content="nb_NO" />
<meta property="og:url" content="http://www.mayasart.net/" />
<meta property="og:title" content="Foto på lerret" />
<meta property="og:description" content="Skapargleda i fargar på lerret, også med synål og stikkepinnar. 
Trykk av åttbladroser i hardangersaum i ulike fargar og storleikar

Velkommen til å kikke inn på nettbutikken  - 
Foto og trykk på lerret, aluplate eller plexiglas
Trykka vert laga av fotograf Senderud - Strikkegnomer - 
Oppskrifter..." />
<meta property="og:image:url" content="https://cdn.simp

In [104]:
from bs4 import BeautifulSoup
content = BeautifulSoup(getContent(urls[0]), 'html.parser')
body = content.find('body')
content = body.findChildren()

In [105]:
content_string = ''.join(str(content))

In [106]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [115]:
import re
content_string = strip_tags(content_string)

content_string = re.sub(r'[^\x00-\x7F]+',' ', content_string)
tags = ['\n','\t','\r','\xa0','word-wrap:break-word;', '.wordwrapfix','{','}']
for tag in tags:
    content_string = content_string.replace(tag, ' ')
    
content_string = re.sub('[ +]', '', content_string)

In [116]:
content_string = content_string.strip()

In [117]:
content_string

'[MenyMayasartAkrylmalerittebladroserKursInfokursMesserogutstillingKontaktOmmeg:GjestebokLinkNettbutikkAdventskalendarStrikkegnomerHardangersaumkortttebladroserFotoplerretIamIexsistOriginalMaleriBunadskjorteposarMayasartKunstogdesignSkapargledaifargarplerret,ogsmedsynlogstikkepinnar.TrykkavttbladroserihardangersaumiulikefargarogstorleikarVelkommentilkikkeinnpnettbutikkenFotoplerretFotoogtrykkplerret,aluplateellerplexiglasTrykkavertlagaavfotografSenderudStrikkegnomerOppskriftertilstrikkegnomerInettbutikkenfinnduoppskriftertilulikestrikkegnomersomStmosane,Smgnomane,Strikkegnomer,FlaskeskjulerogEggevarmere.KunstvandringKunstvandringVossstartaroppmedKunstvandringpinsen2019ogskalblieintradisjonkvartrframover.Kunstvandringerfellesopningstideravverkstader,galleriogatleier,einellerfleiredagar.KunstogkunsthandtverkararfrVaksdal,Voss.UlvikogAurlandheldopeipinsen.Opningstiderogkartoverdeiulikestadanekjemsnart.Sjwww.kunstvandringvoss.comellerFb:KunstvandringVoss,formeirinformasjonetterkvartsomdetv